# Image Combing Notebook 
This notebook aims to retrieve images from the internet for the purpose of dataset creation due to the scarcity of accessible, quality food dataset

## Import statements

In [ ]:
import os
import requests
import configparser
import urllib.request
from bs4 import BeautifulSoup

## Read configs

In [ ]:
config = configparser.ConfigParser()
config.read('configs.ini')
api_key = config['API']['custom_search_api_key']
cx = config['API']['custom_search_cx']
user_agent = config['UserAgents']['user_agent']
geolocation = config['API']['geolocation']
host_language = config['API']['host_language']

## Set up a Custom Search JSON API clien

In [ ]:
service = build("customsearch", "v1", developerKey=api_key)

## Function Declarations

In [ ]:
def get_soup(url, header):
    return BeautifulSoup(requests.get(url, headers=header).text, 'html.parser')

def retrieve_images(search_query, num_images, user_agent):
    # NOT IMPLEMENTED
    pass

## Main Program

### Define params

In [ ]:
target_food = 'hainanese chicken rice'
search_queries = ['Singaporean chicken rice', 'Malaysian chicken rice', 'chinese chicken rice']
no_of_imgs = 190
#This refers to the number of images you already have. 
#So if you already have 30 images, this value will be 30
# Note that the api really only allows you to fetch 200 images, even though the documentation says 100 images of results.
start_num = 200
# download_directory = f'../data/{search_query.replace(" ", "_")}' # Replace directory with desired path
download_directory = f'../data/{target_food.replace(" ", "_")}' # Replace directory with desired path


In [ ]:
images = []
for search_query in search_queries:
    images = fetch_images(cx, search_query, no_of_imgs, geolocation, host_language)
    image_links = [image['link'] for image in images]
    download_images(image_links, download_directory, start_num)
    start_num += no_of_imgs

### Call fetch function

In [ ]:
images = fetch_images(cx, search_query, no_of_imgs, geolocation, host_language)

### Retrieve images links

In [ ]:
image_links = [image['link'] for image in images]

### Download the images

In [ ]:
download_images(image_links, download_directory, start_num)